# Pathology prediction (Enlarged_CM)


@References : Soenksen, L.R., Ma, Y., Zeng, C. et al. Integrated multimodal artificial intelligence framework for healthcare applications. npj Digit. Med. 5, 149 (2022). https://doi.org/10.1038/s41746-022-00689-4

In this notebook, the task is to predict the Enlarged Cardiomediastinum pathology using the CSV embeddings file



## Introduction


Radiology notes were processed to determine if each of the pathologies was explicitly confirmed as present (value = 1), explicitly confirmed as absent (value = 0), inconclusive in the study (value = −1), or not explored (no value).

Selected samples : 0 or 1 values (removing the rest from the training and testing data).

Excluded variables : the unstructured radiology notes component (E_rad ) from the allowable input to avoid potential overfitting or misrepresentations of real predictive value.

The model is based on binary classification for each target chest pathology.

Final sample size for the Enlarged Cardiomediastinum pathology is : (N =  3206)

#### Imports

In [1]:
import os
os.chdir('../')

from pandas import read_csv

from src.data import constants
from src.data.dataset import HAIMDataset
from src.evaluation.pycaret_evaluator import PyCaretEvaluator
from src.utils.metric_scores import *

#### Read data from local source



In [2]:
df = read_csv(constants.FILE_DF, nrows=constants.N_DATA)

#### Create a custom dataset for the HAIM experiment


Build the target column for the task at hand, set the dataset specificities:  the ``haim_id`` as a ``global_id``, use all sources except ``radiology notes`` 

In [3]:
dataset = HAIMDataset(df,  
                      constants.CHEST_PREDICTORS, 
                      constants.ALL_MODALITIES, 
                      constants.ENLARGED_CARDIOMEDIASTINUM,
                      constants.IMG_ID, 
                      constants.GLOBAL_ID)

#### Set hyper-parameters

In [4]:
# Define the grid oh hyper-parameters for the tuning
grid_hps = {'max_depth': [5, 6, 7, 8],
            'n_estimators': [200, 300],
            'learning_rate': [0.3, 0.1, 0.05],
            }

### Model training and predictions using an XGBClassifier model with GridSearchCV and Hyperparameters optimization


The goal of this section of the notebook is to compute the following metrics:

``ACCURACY_SCORE, BALANCED_ACCURACY_SCORE, SENSITIVITY, SPECIFICITY, AUC, BRIER SCORE, BINARY CROSS-ENTROPY``


The
hyperparameter combinations of individual XGBoost models were
selected within each training loop using a ``fivefold cross-validated
grid search`` on the training set (80%). This XGBoost ``tuning process``
selected the ``maximum depth of the trees (5–8)``, the number of
``estimators (200 or 300)``, and the ``learning rate (0.05, 0.1, 0.3)``
according to the parameter value combination leading to the
highest observed AUROC within the training loop 


As mentioned previously, all XGBoost models were trained ``five times with five different data splits`` to repeat the
experiments and compute average metrics 


```Refer to page 8 of study``` : https://doi.org/10.1038/s41746-022-00689-4

In [5]:
# Initialize the PyCaret Evaluator
evaluator = PyCaretEvaluator(dataset=dataset, target="EnlargedCardiomediastinum", experiment_name="CP_EnlargedCardiomediastinum", filepath="./results/enlargedcardiomediastinum")

# Model training and results evaluation
evaluator.run_experiment(
    train_size=0.8,
    fold=5,
    fold_strategy='stratifiedkfold',
    outer_fold=5,
    outer_strategy='stratifiedkfold',
    session_id=42,
    model='xgboost',
    optimize='AUC',
    custom_grid=grid_hps
)

Outer fold 1/5
Configuring PyCaret for outer fold 1
Creating model xgboost for outer fold 1


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8637,0.8928,0.9608,0.8698,0.9130,0.6020,0.6182
1,0.8293,0.8623,0.9279,0.8550,0.8899,0.5123,0.5209
2,0.8634,0.8887,0.9410,0.8831,0.9111,0.6177,0.6235
3,0.8707,0.8723,0.9706,0.8710,0.9181,0.6159,0.6367
4,0.8634,0.8802,0.9771,0.8592,0.9144,0.5838,0.6145
Mean,0.8581,0.8793,0.9555,0.8676,0.9093,0.5863,0.6028
Std,0.0147,0.0111,0.0184,0.0099,0.0100,0.0390,0.0416


Tuning hyperparameters for model xgboost with custom grid


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8589,0.8958,0.9542,0.8690,0.9097,0.5906,0.6043
1,0.8293,0.8666,0.9213,0.8593,0.8892,0.5189,0.5249
2,0.8610,0.8886,0.9344,0.8851,0.9091,0.6147,0.6188
3,0.8780,0.8746,0.9739,0.8765,0.9226,0.6390,0.6591
4,0.8683,0.8854,0.9804,0.8621,0.9174,0.5987,0.6301
Mean,0.8591,0.8822,0.9528,0.8704,0.9096,0.5924,0.6075
Std,0.0164,0.0104,0.0225,0.0095,0.0114,0.0403,0.0450


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Transformation Pipeline and Model Successfully Saved


...

Tuning hyperparameters for model xgboost with custom grid


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8735,0.8960,0.9641,0.8780,0.9190,0.6330,0.6477
1,0.8662,0.8953,0.9510,0.8792,0.9137,0.6184,0.6279
2,0.8439,0.8652,0.9311,0.8685,0.8987,0.5601,0.5666
3,0.8634,0.8911,0.9444,0.8811,0.9117,0.6122,0.6194
4,0.8415,0.8925,0.9346,0.8640,0.8980,0.5452,0.5537
Mean,0.8577,0.8880,0.9451,0.8742,0.9082,0.5938,0.6031
Std,0.0127,0.0116,0.0118,0.0067,0.0084,0.0346,0.0364


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Transformation Pipeline and Model Successfully Saved


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.8440,0.8545,0.9518,0.8550,0.9008,0.5407,0.5581


Final metrics table:
     Metric     Mean   Std Dev
0  Accuracy  0.85646  0.015014
1       AUC  0.88250  0.019545
2    Recall  0.95360  0.010194
3     Prec.  0.86692  0.009822
4        F1  0.90820  0.009547
5     Kappa  0.58264  0.044186
6       MCC  0.59686  0.044837
